In [66]:
import pandas as pd
from neo4j import GraphDatabase

# Connect to Neo4J and download the vertices and edges of the graph

In [67]:
URI = "bolt://localhost:7687"
USERNAME = "neo4j"
PASSWORD = "123"

driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))

In [68]:
nodes_list = []
relations_list = []

with driver.session() as session:
    result = session.run("MATCH (N) RETURN N")
    nodes_list = result.data()
    
    result = session.run("MATCH (A)-[:COMMON_STUDENT]-(B) RETURN A,B")
    relations_list = result.data()

### Manipulate the dicts to generate an adjacent matrix on pandas

In [69]:
nodes_list = [i["N"]["name"] for i in nodes_list]

In [70]:
dict_pandas = {}
qtd_objects = len(nodes_list)

for item in nodes_list:
    dict_pandas[item] = [0 for i in range(len(nodes_list))]

In [71]:
for key in dict_pandas.keys():
    for rel in relations_list:
        if rel['A']['name'] == key:
            dict_pandas[key][int(rel['B']['name'].split()[-1])] = 1
            

In [72]:
df = pd.DataFrame(dict_pandas)
df

,Course 1,Course 2,Course 3,Course 4,Course 5,Course 6,Course 7,Course 8,Course 9,Course 10,...,Course 91,Course 92,Course 93,Course 94,Course 95,Course 96,Course 97,Course 98,Course 99,Course 0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
97,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
98,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0


# Greedy with Heuristic 1

In [73]:
colors = [1]
result = {}
for key in df.keys():
    result[key] = None

In [74]:
def check_color_availability(key):
    neighbor_colors = []
    neighbors = []
    for i in range(len(df[key])):
        if df[key][i] == 1:
            neighbor_colors.append(result[f"Course {i}"])
    
    safe_color = None
    for item in colors:
        if item not in neighbor_colors:
            safe_color = item
            break
    
    if safe_color is None:
        safe_color = colors[-1] + 1
        colors.append(safe_color)
    
    return safe_color

#### Results: the number of the color for each vertice

In [75]:
for key in df.keys():
    result[key] = check_color_availability(key)

result

{'Course 1': 1,
 'Course 2': 1,
 'Course 3': 1,
 'Course 4': 1,
 'Course 5': 1,
 'Course 6': 2,
 'Course 7': 1,
 'Course 8': 1,
 'Course 9': 3,
 'Course 10': 2,
 'Course 11': 1,
 'Course 12': 2,
 'Course 13': 1,
 'Course 14': 1,
 'Course 15': 2,
 'Course 16': 1,
 'Course 17': 1,
 'Course 18': 1,
 'Course 19': 1,
 'Course 20': 1,
 'Course 21': 2,
 'Course 22': 1,
 'Course 23': 1,
 'Course 24': 2,
 'Course 25': 1,
 'Course 26': 1,
 'Course 27': 1,
 'Course 28': 1,
 'Course 29': 3,
 'Course 30': 2,
 'Course 31': 3,
 'Course 32': 2,
 'Course 33': 1,
 'Course 34': 3,
 'Course 35': 2,
 'Course 36': 2,
 'Course 37': 1,
 'Course 38': 2,
 'Course 39': 1,
 'Course 40': 1,
 'Course 41': 3,
 'Course 42': 2,
 'Course 43': 2,
 'Course 44': 1,
 'Course 45': 1,
 'Course 46': 3,
 'Course 47': 3,
 'Course 48': 3,
 'Course 49': 2,
 'Course 50': 3,
 'Course 51': 2,
 'Course 52': 2,
 'Course 53': 2,
 'Course 54': 4,
 'Course 55': 4,
 'Course 56': 4,
 'Course 57': 2,
 'Course 58': 1,
 'Course 59': 1,
 'Cour

#### Minimal amount of colors required to solve the graph

In [76]:
len(colors)

5

# Greedy with Heuristic 2

In [77]:
degrees = [{"key": key, "degree": sum(df[key])} for key in df.keys()]

In [78]:
def bubbleSort(arr):

    n = len(arr)

    for i in range(n):
        for j in range(0, n - i - 1):
            if arr[j]["degree"] < arr[j + 1]["degree"]:
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
                
bubbleSort(degrees)

In [79]:
colors = [1]
result = {}
for key in dict_pandas.keys():
    result[key] = None

#### Results: the number of the color for each vertice

In [80]:
for key in degrees:
    result[key["key"]] = check_color_availability(key["key"])

result

{'Course 1': 1,
 'Course 2': 2,
 'Course 3': 2,
 'Course 4': 1,
 'Course 5': 3,
 'Course 6': 1,
 'Course 7': 2,
 'Course 8': 1,
 'Course 9': 3,
 'Course 10': 2,
 'Course 11': 3,
 'Course 12': 3,
 'Course 13': 2,
 'Course 14': 2,
 'Course 15': 1,
 'Course 16': 2,
 'Course 17': 1,
 'Course 18': 3,
 'Course 19': 2,
 'Course 20': 3,
 'Course 21': 4,
 'Course 22': 1,
 'Course 23': 1,
 'Course 24': 3,
 'Course 25': 1,
 'Course 26': 3,
 'Course 27': 3,
 'Course 28': 2,
 'Course 29': 2,
 'Course 30': 1,
 'Course 31': 2,
 'Course 32': 1,
 'Course 33': 1,
 'Course 34': 2,
 'Course 35': 2,
 'Course 36': 3,
 'Course 37': 3,
 'Course 38': 2,
 'Course 39': 2,
 'Course 40': 3,
 'Course 41': 1,
 'Course 42': 1,
 'Course 43': 2,
 'Course 44': 2,
 'Course 45': 3,
 'Course 46': 2,
 'Course 47': 2,
 'Course 48': 2,
 'Course 49': 1,
 'Course 50': 4,
 'Course 51': 1,
 'Course 52': 1,
 'Course 53': 2,
 'Course 54': 2,
 'Course 55': 3,
 'Course 56': 3,
 'Course 57': 1,
 'Course 58': 4,
 'Course 59': 4,
 'Cour

#### Minimal amount of colors required to solve the graph

In [81]:
max([i for i in result.values()])

4